<a href="https://colab.research.google.com/github/Santosh-Gupta/Arxiv-Manatee/blob/master/WorkingFullConvertAllPotentialStructuredAbstracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import zipfile
import os
import sys
import pandas as pd
import numpy as np
import gc
import pickle

!pip install fastparquet

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install fastparquet

     |████████████████████████████████| 153kB 2.8MB/s 
     |████████████████████████████████| 61kB 26.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/f3/27/fb/839c776ec8689ff9ee52ad3e91d7d4d848ac6d7545a127d5b0
  Stored in directory: /root/.cache/pip/wheels/be/36/81/0f93ba89a1cb7887c91937948519840a72c0ffdd57cac0ae8f
Successfully built fastparquet thrift


In [0]:
# theBigList =[]
file2C = 'gdrive/My Drive/PotentialStructuredAbstracts/SectionalSummarizationData{:02}.npy'
saver =  'gdrive/My Drive/AbstractsParsed{:02}.parquet.gzip'

In [0]:
listStrings = { 
'\nIntroduction' , '\nCase' , 
'\nLiterature' , '\nBackground',  '\nRelated' , 
'\nMethods' , '\nMethod',
'\nTechniques', '\nMethodology',
'\nResults', '\nResult', '\nExperimental',
'\nExperiments', '\nExperiment',
'\nDiscussion' , '\nLimitations',
'\nConclusion' , '\nConclusions',
'\nConcluding' ,
'Introduction\n' , 'Case\n' , 
'Literature\n' , 'Background\n',  'Related\n' , 
'Methods\n' , 'Method\n',
'Techniques\n', 'Methodology\n',
'Results\n', 'Result\n', 'Experimental\n',
'Experiments\n', 'Experiment\n',
'Discussion\n' , 'Limitations\n',
'Conclusion\n' , 'Conclusions\n',
'Concluding\n' ,
'INTRODUCTION' , 'CASE' , 
'LITERATURE' , 'BACKGROUND',  'RELATED' , 
'METHODS' , 'METHOD',
'TECHNIQUES', 'METHODOLOGY',
'RESULTS', 'RESULT', 'EXPERIMENTAL',
'EXPERIMENTS', 'EXPERIMENT',
'DISCUSSION' , 'LIMITATIONS',
'CONCLUSION' , 'CONCLUSIONS',
'CONCLUDING' ,
'Introduction:' , 'Case:' , 
'Literature:' , 'Background:',  'Related:' , 
'Methods:' , 'Method:',
'Techniques:', 'Methodology:',
'Results:', 'Result:', 'Experimental:',
'Experiments:', 'Experiment:',
'Discussion:' , 'Limitations:',
'Conclusion:' , 'Conclusions:',
'Concluding:' ,
}

pat = '|'.join(listStrings)

In [0]:
def parseDataframe(inputDF):
            # first we build a big regex pattern
    # find all keywords in the series
    new_df = theBigList.paperAbstract.str.findall(pat)

    # find all the chunks by splitting the text with the found keywords
    chunks = pd.concat([theBigList.paperAbstract.iloc[[i]].str.split('|'.join(new_df.iloc[i]), expand=True) 
                 for i in range(len(theBigList))]).stack()
    
    # stack the keywords:
    keys = np.concatenate(new_df.values) # Flatten the keywords array
    values = chunks.groupby(level=0).shift(-1).dropna().values 

    df = theBigList[["pdfUrls",	"s2PdfUrl",	"s2Url"]].copy() # Copy the dataframe
    labels = df.loc[df.index.repeat(new_df.apply(len))].reset_index(drop=True) # repeat urls based on how many "keys" each paper has
    
    # Or using 3 columns like the example
    returnDF = pd.DataFrame({'C': keys, 'D': values, 'pdfUrls': labels['pdfUrls'], 's2PdfUrl': labels['s2PdfUrl'], 's2Url': labels['s2Url']})
    
    returnDF['D'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
    returnDF['C'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
    returnDF['C'].replace(':', '', regex=True, inplace=True) 
    returnDF['D'].replace('S ', '', regex=True, inplace=True)
    
    return returnDF

In [0]:
for i in range(0, 47):
    gc.collect()
    print('step ', i)
    
    gc.collect()

    combC = np.load(file2C.format(i) , allow_pickle=True)

    theBigList = pd.DataFrame(combC)
    theBigList.columns =  [ 'title', 'paperAbstract', 'pdfUrls', 's2PdfUrl' , 's2Url' ]

    finishedDF = parseDataframe(theBigList)

    finishedDF.to_parquet( saver.format(i) , compression='gzip', engine = 'fastparquet' , index = False )  
    del combC

step  0
step  1
step  2
step  3
step  4
step  5
step  6
step  7
step  8
step  9
step  10
step  11
step  12
step  13
step  14
step  15


In [0]:
finishedDF.tail(100)

,C,D,pdfUrls,s2PdfUrl,s2Url
100289,BACKGROUND,"Thyroid autoantibodies (ThyAb) and subclinical hypothyroidism occur more frequently in pregnant women with insulin-dependent diabetes mellitus than in healthy pregnant women. Few studies have investigated the presence of ThyAb in women with gestational diabetes mellitus (GDM), and no significant association between diabetes in pregnancy and thyroid function has been reported. OBJECTIVE To assess the thyroid biochemical profile and estimate the prevalence of ThyAb in a group of pregnant women at increased risk of GDM due to family and personal risk factors, and to investigate the relationship between a positive family history of diabetes or thyroid diseases and the eventual presence of ThyAb during pregnancy.",[https://eje.bioscientifica.com/downloadpdf/journals/eje/143/6/741.pdf],,https://semanticscholar.org/paper/9096f63926355e9a7a8ace9c267d5f9b7465a56b
100290,METHOD,"Oral glucose tolerance, serum ThyAb and thyroid function were evaluated in 181 pregnant women with increased risk for GDM (study group). Seventeen healthy pregnant women without risk factors for GDM and with a normal glucose tolerance were recruited as controls.",[https://eje.bioscientifica.com/downloadpdf/journals/eje/143/6/741.pdf],,https://semanticscholar.org/paper/9096f63926355e9a7a8ace9c267d5f9b7465a56b
100291,RESULT,"The women who developed GDM showed a mean free thyroxine concentration significantly lower than that observed in the healthy pregnant women and in those with impaired gestational glucose tolerance and normal glucose tolerance. Twenty-nine of the 181 women in the study group (16%) were ThyAb positive. However, the risk of being ThyAb positive during pregnancy was three times greater in the women with positive family history of both diabetes mellitus and thyroid disease than in those with no family history of these conditions.",[https://eje.bioscientifica.com/downloadpdf/journals/eje/143/6/741.pdf],,https://semanticscholar.org/paper/9096f63926355e9a7a8ace9c267d5f9b7465a56b
100292,CONCLUSION,"This study showed that women with increased risk of GDM, mostly those with family history of diabetes mellitus and thyroid disease, also have an increased risk of being ThyAb positive during pregnancy. It also highlighted the importance of evaluating thyroid function in pregnant women with impaired glucose tolerance, in view of their increased risk of subclinical hypothyroidism.",[https://eje.bioscientifica.com/downloadpdf/journals/eje/143/6/741.pdf],,https://semanticscholar.org/paper/9096f63926355e9a7a8ace9c267d5f9b7465a56b
100293,INTRODUCTION,Anaplastic large cell lymphoma is a subset of non-Hodgkin lymphoma and an unusual disease in children.,"[http://cdn.neoscriber.org/cdn/serve/8e/f9/8ef94cdc7dcbae3129bd49eba6a18bba43b3abf7/ijcp-08-3438.pdf, https://cdn.neoscriber.org/cdn/serve/2e/55/2e5595b79ba9d78c696d7c76aed161a720146a41/ijcp-08-3438.pdf]",,https://semanticscholar.org/paper/2660d7b0e73598b9b6370d7668063792a6ad51bc
100294,CASE,PRESENTATION Herein we have reported a 7- year- old girl with a large necrotic skin ulcer on the chest caused by systemic form of anaplastic large-cell lymphoma and simultaneous active toxoplasmosis diagnosed by PCR on lymph node specimen. There were few reports showing a role for toxoplasma infection to cause some malignancies such as lymphoma in adults.,"[http://cdn.neoscriber.org/cdn/serve/8e/f9/8ef94cdc7dcbae3129bd49eba6a18bba43b3abf7/ijcp-08-3438.pdf, https://cdn.neoscriber.org/cdn/serve/2e/55/2e5595b79ba9d78c696d7c76aed161a720146a41/ijcp-08-3438.pdf]",,https://semanticscholar.org/paper/2660d7b0e73598b9b6370d7668063792a6ad51bc
100295,CONCLUSION,"Based to our knowledge, this has been the first report of simultaneous systemic anaplastic large cell lymphoma and active toxoplasmosis, documented by positive PCR on tissue biopsy in a child. This case report has suggested more attention to the accompanying Toxoplasma gondii infection as a probable cause of some types of lymphomas.","[

In [0]:
pd.set_option('max_colwidth', 3000)

In [0]:
# first we build a big regex pattern
pat = '|'.join(listStrings)

# find all keywords in the series
new_df = theBigList.paperAbstract.str.findall(pat)

# find all the chunks by splitting the text with the found keywords
chunks = pd.concat([theBigList.paperAbstract.iloc[[i]].str.split('|'.join(new_df.iloc[i]), expand=True) 
             for i in range(len(theBigList))]).stack()

In [0]:
# stack the keywords:
keys = np.concatenate(new_df.values) # Flatten the keywords array
values = chunks.groupby(level=0).shift(-1).dropna().values 

df = theBigList[["pdfUrls",	"s2PdfUrl",	"s2Url"]].copy() # Copy the dataframe
labels = df.loc[df.index.repeat(new_df.apply(len))].reset_index(drop=True) # repeat urls based on how many "keys" each paper has

In [0]:
# output df
pd.DataFrame({'C': keys, 'D': values, 'pdfUrls': labels['pdfUrls'], 's2PdfUrl': labels['s2PdfUrl'], 's2Url': labels['s2Url']})

,C,D,pdfUrls,s2PdfUrl,s2Url
0,BACKGROUND,"\nRecent attention has focused on the expression of integrin molecules within the endometrium, and their relation to infertility.\n\n\nOBJECTIVE\nThe present prospective study was undertaken to determine whether the endometrium of women with unexplained infertility differs in the expression of very late activation antigens (VLA) from the endometrium of normal fertile women.\n\n\n",[http://iji.sums.ac.ir/article_16996_d18996e768c77622f726738dc56303bf.pdf],http://pdfs.semanticscholar.org/7452/1cb86ce802a28983d030b570588b353538df.pdf,https://semanticscholar.org/paper/74521cb86ce802a28983d030b570588b353538df
1,METHOD,"S\nThirty samples of endometrial biopsies from hysterectomies with nonendometrial pathology and 28 endometrial samples by uterine curetting from infertile women in secretary phase at implantation time were collected, stained with three monoclonal antibodies against beta1 integrin subunits including VLA-1 to VLA-3 by immunohistochemical technique and then assessed semiquantitively by microscope. Chi-Square test was used to compare the expression of VLA antigens on epithelial cells, stromal cells, lymphocytes and vessels within endometrial tissues between two groups.\n\n\n",[http://iji.sums.ac.ir/article_16996_d18996e768c77622f726738dc56303bf.pdf],http://pdfs.semanticscholar.org/7452/1cb86ce802a28983d030b570588b353538df.pdf,https://semanticscholar.org/paper/74521cb86ce802a28983d030b570588b353538df
2,RESULTS,"\nThe results showed that most VLA integrins were present in fertile and infertile endometrium tissues. There were similarities and differences in the expression of VLA molecules in different compartments. VLA-2, VLA-3 expression on endometrial compartments showed an unaltered pattern of staining during the putative window of implantation in either fertile or infertile women with no significant differences (P-value> 0.5). VLA-1 expression on endometrial compartments changed in fertile and unexplained infertile women, the differences were related to the presence or lack of the molecules on epithelial and stromal cells respectively.\n\n\n",[http://iji.sums.ac.ir/article_16996_d18996e768c77622f726738dc56303bf.pdf],http://pdfs.semanticscholar.org/7452/1cb86ce802a28983d030b570588b353538df.pdf,https://semanticscholar.org/paper/74521cb86ce802a28983d030b570588b353538df
3,CONCLUSION,"\nDifferences may explain causes of unexplained infertility, and suggests that certain integrins may participate in the cascade of molecular events leading to successful implantation and early placental development which requires more investigations.",[http://iji.sums.ac.ir/article_16996_d18996e768c77622f726738dc56303bf.pdf],http://pdfs.semanticscholar.org/7452/1cb86ce802a28983d030b570588b353538df.pdf,https://semanticscholar.org/paper/74521cb86ce802a28983d030b570588b353538df
4,BACKGROUND,"\nCardiopulmonary bypass (CPB) is usually associated with inflammatory response that leads to various degrees of organ dysfunction in multiple systems, including lung injury. Our previous study showed that transforming growth factor beta1 (TGFβ1) was involved in CPB-induced lung injury. N-acetylcysteine (NAC) is an antioxidant and is able to prevent CPB-induced pneumocyte apoptosis through scavenging radical. Therefore, we investigated whether NAC may attenuate CPB-induced lung injury by inhibiting TGFβ1 expression.\n\n\n",[http://download-redirector.springer.com/redirect?contentType=pdf&ddsId=art:10.1186/1749-8090-8-107&originUrl=http://cardiothoracicsurgery.biomedcentral.com/article/10.1186/1749-8090-8-107],http://pdfs.semanticscholar.org/be46/a183b2f699e04915e9caee408766844ac10a.pdf,https://semanticscholar.org/paper/2ce19a5e41aa5bfbceedb85139ceb934dca920fa
5,METHOD,"S\nFifty-four 18 to 24-month-old mongrel dogs (15-16 kg) were randomly divided into control group, CPB group and NAC group (n = 18). Six dogs in each group were killed prior to, as well as 30 and 60 minutes after the operation (T0, T1 and T2). Lung injur

In [0]:
# Or using 3 columns like the example
pd.DataFrame({'C': keys, 'D': values, 'E': list(map(lambda x: [x], labels.values))})

,C,D,E
0,BACKGROUND,"\nRecent attention has focused on the expression of integrin molecules within the endometrium, and their relation to infertility.\n\n\nOBJECTIVE\nThe present prospective study was undertaken to determine whether the endometrium of women with unexplained infertility differs in the expression of very late activation antigens (VLA) from the endometrium of normal fertile women.\n\n\n","[[[http://iji.sums.ac.ir/article_16996_d18996e768c77622f726738dc56303bf.pdf], http://pdfs.semanticscholar.org/7452/1cb86ce802a28983d030b570588b353538df.pdf, https://semanticscholar.org/paper/74521cb86ce802a28983d030b570588b353538df]]"
1,METHOD,"S\nThirty samples of endometrial biopsies from hysterectomies with nonendometrial pathology and 28 endometrial samples by uterine curetting from infertile women in secretary phase at implantation time were collected, stained with three monoclonal antibodies against beta1 integrin subunits including VLA-1 to VLA-3 by immunohistochemical technique and then assessed semiquantitively by microscope. Chi-Square test was used to compare the expression of VLA antigens on epithelial cells, stromal cells, lymphocytes and vessels within endometrial tissues between two groups.\n\n\n","[[[http://iji.sums.ac.ir/article_16996_d18996e768c77622f726738dc56303bf.pdf], http://pdfs.semanticscholar.org/7452/1cb86ce802a28983d030b570588b353538df.pdf, https://semanticscholar.org/paper/74521cb86ce802a28983d030b570588b353538df]]"
2,RESULTS,"\nThe results showed that most VLA integrins were present in fertile and infertile endometrium tissues. There were similarities and differences in the expression of VLA molecules in different compartments. VLA-2, VLA-3 expression on endometrial compartments showed an unaltered pattern of staining during the putative window of implantation in either fertile or infertile women with no significant differences (P-value> 0.5). VLA-1 expression on endometrial compartments changed in fertile and unexplained infertile women, the differences were related to the presence or lack of the molecules on epithelial and stromal cells respectively.\n\n\n","[[[http://iji.sums.ac.ir/article_16996_d18996e768c77622f726738dc56303bf.pdf], http://pdfs.semanticscholar.org/7452/1cb86ce802a28983d030b570588b353538df.pdf, https://semanticscholar.org/paper/74521cb86ce802a28983d030b570588b353538df]]"
3,CONCLUSION,"\nDifferences may explain causes of unexplained infertility, and suggests that certain integrins may participate in the cascade of molecular events leading to successful implantation and early placental development which requires more investigations.","[[[http://iji.sums.ac.ir/article_16996_d18996e768c77622f726738dc56303bf.pdf], http://pdfs.semanticscholar.org/7452/1cb86ce802a28983d030b570588b353538df.pdf, https://semanticscholar.org/paper/74521cb86ce802a28983d030b570588b353538df]]"
4,BACKGROUND,"\nCardiopulmonary bypass (CPB) is usually associated with inflammatory response that leads to various degrees of organ dysfunction in multiple systems, including lung injury. Our previous study showed that transforming growth factor beta1 (TGFβ1) was involved in CPB-induced lung injury. N-acetylcysteine (NAC) is an antioxidant and is able to prevent CPB-induced pneumocyte apoptosis through scavenging radical. Therefore, we investigated whether NAC may attenuate CPB-induced lung injury by inhibiting TGFβ1 expression.\n\n\n","[[[http://download-redirector.springer.com/redirect?contentType=pdf&ddsId=art:10.1186/1749-8090-8-107&originUrl=http://cardiothoracicsurgery.biomedcentral.com/article/10.1186/1749-8090-8-107], http://pdfs.semanticscholar.org/be46/a183b2f699e04915e9caee408766844ac10a.pdf, https://semanticscholar.org/paper/2ce19a5e41aa5bfbceedb85139ceb934dca920fa]]"
5,METHOD,"S\nFifty-four 18 to 24-month-old mongrel dogs (15-16 kg) were randomly divided into control group, CPB group and NAC group (n = 18). Six dogs in each group were killed prior to, as well as 30 and 60 minutes after the operation (T0, T1 